In [9]:
"""
## JMP Model - multicountry version, any C and J
## Created Feb 7,2025
## Updated Feb 26
## Author VG

This file solves the equilibrium prices, allocations, and wages, GIVEN wedges and policy

For any number of countries and sectors. 

"""



import matplotlib.pyplot as plt
import matplotlib.axes as axs
import numpy as np
import pandas as pd
import networkx as nx 
import textwrap
import seaborn as sns
import random
from scipy.optimize import fsolve
from scipy.optimize import minimize

maindir="/Users/VanyaG1/Dropbox/VANYA/PhD/PROJECTS/Sector Centrality/JMP Model/py/"



from Model_func_2_solve_eq_2country import *  
from Model_func_2_solve_eq_multicountry import *  
#model function: Solve price system and demand system as functions of prices. Then solve for wages using TB condition


### NB on matrix conventions
#1. First set of index are the buyer. Second set are the seller
#2. Untransformed matrices: rows correspond to first sub and super scripts - BUYER. Columns correspond to second sub/superscripts - SELLER


In [ ]:
"""
Solve for equilibrium given wedges and policy
"""

#params 
C=10 #number of countries
J=20 #number of industries
epsilon=5 #production side elasticity of substitution
sigma=4

params=[C,J,epsilon,sigma]

L = np.arange(10, C+1 * 10, 10)
for rand in range(0,35):
    
    #sample values
    np.random.seed(rand)

    # CES weights
    alp = np.random.uniform(0, 0.1, size=(C*J,1)) # weight on intermediates overall
    gam = np.random.uniform(0, 0.5, size=(C*J, C*J)) # weights on intermediates by sector: row i, col j is weight on input sector j for buying sector i
    gam = gam/gam.sum(axis=1, keepdims=True) # normalize so that the rows sum to 1

    #set sample wedges
    chi_mat=np.random.uniform(0, 0.5, size=(C*J, C*J)) 

    #set sample policy
    tau_mat=np.random.uniform(0, 0.5, size=(C*J, C*J)) 
    tau_L=np.random.uniform(0, 0.5, size=(C*J, 1))

    #Guess initial wage C-1 countries, c=1 wage normalized
    init_guess=2*np.ones(C-1).reshape(-1,1)
    solve_eq=1
    
    #Run function to solve for equilibrium wages
    result = fsolve(solver,init_guess,args=(params,L,alp,gam,chi_mat,tau_mat,tau_L,solve_eq))
    #model function: Solve price system and demand system as functions of prices. 
    #Then solve for wages using TB condition. Returns wage for c=2, wage for c=1 normalized to 1

    W_out=np.concatenate((np.array([1]),result), axis=0).reshape(-1,1)
    
    #run solver function again at the equilibrium W to pull out consumption (add any var of interest here)
    solve_eq=0
    consumption  = solver(result,params,L,alp,gam,chi_mat,tau_mat,tau_L,solve_eq)
    
    
    print("equilibrium wages: "+str(W_out))